In [ ]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()
ngs.ngsglobals.msg_level = 4
# ngs.SetHeapSize(100*1000*1000)

In [ ]:
%run J.ipynb

In [ ]:
from bhdata import BHCurves
fun_dw  = BHCurves(-4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

In [ ]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

p = 2

H1 = ngs.H1(mesh, order = p, dirichlet = 'ambient_face')
u,v = H1.TnT()


# Nonlinear:
maxit = 10000
tol2 = 1e-8
regh = 1e-12

psi = ngs.GridFunction(H1)
H = ngs.grad(psi) + Hs
normH = ngs.sqrt(H*H + regh)

cf_coenergy = mesh.MaterialCF({linear: mu0/2*H*H, nonlinear: fun_w(normH)}, default = mu0/2*H*H).Compile()
def fun_W():
    # with ngs.TaskManager(): 
    res = ngs.Integrate(cf_coenergy, mesh)
    return res

cf_rhs = mesh.MaterialCF({linear: mu0, nonlinear: fun_dw(normH)/normH}, default = mu0).Compile()
rhs = ngs.LinearForm(cf_rhs*H*ngs.grad(v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    # with ngs.TaskManager(): 
    rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

HHt = ngs.CF((H[0]*H[0], H[0]*H[1], H[0]*H[2],
              H[1]*H[0], H[1]*H[1], H[1]*H[2],
              H[2]*H[0], H[2]*H[1], H[2]*H[2]), dims=(3,3))


fun1 = fun_dw(normH)/normH
fun2 = (fun_ddw(normH) - fun_dw(normH)/normH)/(normH*normH)

cf_iter = mesh.MaterialCF({linear: mu0*Id, nonlinear: fun1*Id + fun2*(HHt)}, default = mu0*Id).Compile()

K_iter = ngs.BilinearForm(H1)
K_iter += ((cf_iter*ngs.grad(u))*ngs.grad(v))*ngs.dx
# K_iter += (cf_iter[0]*gU[0]*gV[0] + cf_iter[1]*gU[1]*gV[0] + cf_iter[2]*gU[2]*gV[0] +\
#            cf_iter[3]*gU[0]*gV[1] + cf_iter[4]*gU[1]*gV[1] + cf_iter[5]*gU[2]*gV[1] +\
#            cf_iter[6]*gU[0]*gV[2] + cf_iter[7]*gU[1]*gV[2] + cf_iter[8]*gU[2]*gV[2])*ngs.dx

# K_iter += ((cf_iter*ngs.grad(u))*ngs.grad(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    # with ngs.TaskManager(): 
    K_iter.Assemble()
    return K_iter

In [ ]:
with ngs.TaskManager():
    print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,H1.ndof, "DOFs.\n ")

    with ngs.TaskManager(): psi.Set(ngs.CF((0)))

    du = ngs.GridFunction(H1)
    uo = ngs.GridFunction(H1)
    wo = 1e12

    for it in range(1,maxit+1):
        
        tic()
        # w  = fun_W()
        res = ngs.Integrate(cf_coenergy, mesh)
        w = res
        tm10 = toc()

        tic()
        # dw = fun_dW()
        rhs.Assemble()
        dw = rhs
        tm11 = toc()

        tic()
        # da = fun_ddW()
        K_iter.Assemble()
        da = K_iter
        tm12 = toc()
        
        tic()
        # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
        # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
        with ngs.TaskManager():
            iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-2,  maxiter = maxit)
            # iterativeSolver = CGSolver(K_iter.mat, freedofs = H1.FreeDofs(), tol  = 1e-2,  maxiter = maxit)
            du.vec.data = iterativeSolver * dw.vec 
        
        if len(iterativeSolver.residuals) == maxit: print("... Success!")
        # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
        tm2 = toc()

        nrm = ngs.InnerProduct(du.vec,dw.vec)
        
        if it == 1:
            nrm0 = nrm
        
        if nrm/nrm0<tol2:
            print("converged to desired tolerance")
            break
        elif abs(wo-w) < tol2*1e-2:
            print("stopped early due to stagnation")
            break
        else:
            # linesearch
            uo.vec.data = psi.vec.data
            wo = w
            alpha = 1
            for init in range(1,21):
                psi.vec.data -= alpha*du.vec.data
                wn = fun_W()
                if wn < w - alpha*0.1*nrm:
                    print("Iter: %2d | assem : %.2fs,%.2fs,%.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm10,tm11,tm12,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                    break
                else:
                    alpha = alpha/2
                    psi.vec.data = uo.vec.data

In [ ]:
L2 = ngs.VectorL2(mesh, order=2)
Bl2 = ngs.GridFunction(L2)
B = fun_dw(normH)/normH*H

with ngs.TaskManager(): Bl2.Set(B)
Bl2np = Bl2.vec.FV().NumPy()


# import matplotlib.pyplot as plt
# import numpy as np
max(abs(Bl2np))

In [ ]:
Draw(B, mesh, clipping={"z":-1, "dist":0.064})